# Battle of Neighbourhood - Capstone Project Week1

Recommender system for finding suitable location to start Pizza places in Toronto.

## Introduction: Business Problem

Toronto is the capital of the province of Ontario in southeastern Canada. It is the most populous city in Canada, a multicultural city, and the country’s financial and commercial centre.
Being the largest city in Canada with an estimated population of over 6 million, there is no doubt about the diversity of the population. Multiculturalism is seen through the various neighbourhoods including; Chinatown, Corso Italia, Little India, Kensington Market, Little Italy, Koreatown and many more.
Downtown Toronto being the hub of interactions between ethnicities brings many opportunities for entrepreneurs to start or grow their business. It is a place where people can try the best of each culture, either while they work or just passing through. Toronto is well known for its great food.


The objective of this project is to  help a prospective investor to find the best location for the opening of a Pizza restaurant/takeout for locals living in Toronto, Canada. 
Pizza is one of the most poplar dishes in any city , kind of business easy to start up with a moderate  investment, great chance to expand in futre, there are numerous opportunities to open a new Pizza place. Through this project, we will find the most suitable location for an entrepreneur to open a new Pizza restaurant/take out in Toronto, Canada.


## Data

We require geographical location data for Toronto . Postal codes in each city serve as a starting point. Using Postal codes we can find out the neighborhoods, boroughs, venues and their most popular venue categories.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto.
Data Link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M                   
From week 3. Dataset consisting of latitude and longitude, zip codes.                        

Two of the neighbouring boroughs chosen for  comparison are North York and Downtown Toronto.                                                                                                                                                                                
-Foursquare API Data:

We will need data about different venues in different neighbourhoods of that specific borough. In order to gain that information we will use "Foursquare" locational information. Foursquare is a location data provider with information about all manner of venues and events within an area of interest. Such information includes venue names, locations, menus and even photos. As such, the foursquare location platform will be used as the sole data source since all the stated required information can be obtained through the API.

After finding the list of neighbourhoods, we then connect to the Foursquare API to gather information about venues inside each and every neighbourhood. For each neighbourhood, we have chosen the radius to be 500 meters.

The data retrieved from Foursquare contained information of venues within a specified distance of the longitude and latitude of the postcodes. The information obtained per venue as follows:

Neighbourhood : Name of the Neighbourhood
Neighbourhood Latitude : Latitude of the Neighbourhood                                     
Neighbourhood Longitude : Longitude of the Neighbourhood                                   
Venue : Name of the Venue                                                                   
Venue Latitude : Latitude of Venue                                                         
Venue Longitude : Longitude of Venue                                                       
Venue Category : Category of Venue                                                         
Based on all the information collected for both boroughs in Tornto , we have sufficient data to build our model. We cluster the neighbourhoods together based on similar venue categories. We then present our observations and findings. Using this data, our stakeholders can take the necessary decision.




Lets import the Libraries:


In [4]:
import numpy as np
import pandas as pd
import requests
import folium
import json 
from pandas.io.json import json_normalize

from sklearn.cluster import KMeans
import matplotlib.cm as cm


import matplotlib.colors as colors

### Retrieve Data From Wikipedia

Read data from Wikipedia and scrape the page using BeautifulSoup

In [5]:
import requests
from bs4 import BeautifulSoup

neighbour_url = requests.get('https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=942851379')
soup = BeautifulSoup(neighbour_url.text,'lxml')

neighbour_table = soup.find_all('table')[0]

Create data frame from html.

In [6]:
df = pd.read_html(str(neighbour_table))
df=pd.DataFrame(df[0]) 


## Data Wrangling

Drop the column with Borough not assigned

In [8]:
df.replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=["Borough"], axis=0, inplace=True)

# reset index, because we droped rows
df.reset_index(drop=True, inplace=True)
df

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


In [9]:
df = df.groupby(['Postcode','Borough'], sort=False).agg(', '.join)
df.reset_index(inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Downtown Toronto,Queen's Park


### Adding Geographical Information

Adding columns for longitude and lattitude

In [11]:
import pgeocode

# retrieve the latitude/longitude from a postal code in Canada 'ca'
nomi_ca = pgeocode.Nominatim('ca')

latitude = []
longitude = []


for index, row in df.iterrows():
    location = nomi_ca.query_postal_code(row[0])  # row[0] represents Postal Code value
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    
# we put the result of the loop in new columns 'latitude' and 'longitude'
df['Latitude'] = latitude
df['Longitude'] = longitude


# pb with Canada Post Gateway Processing Centre > need to do the query manually
df.loc[df['Neighbourhood'] == "Canada Post Gateway Processing Centre", ['Latitude', 'Longitude']] = [43.636966,-79.615819]




In [12]:
df.head()



,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7223,-79.4504
4,M7A,Downtown Toronto,Queen's Park,43.6641,-79.3889


### Explore Toronto Neighbourhood

In [13]:
from geopy import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
t_latitude= location.latitude
t_longitude = location.longitude



In [40]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[t_latitude, t_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_toronto)  
    
map_toronto


### Explore the Location North York

In [15]:
ny_data = df[df['Borough'] == 'North York'].reset_index(drop=True)
ny_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.3300
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7223,-79.4504
3,M3B,North York,Don Mills North,43.7450,-79.3590
4,M6B,North York,Glencairn,43.7081,-79.4479


### Lets visualise the area North York

In [16]:
address = 'North York, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
ny_latitude= location.latitude
ny_longitude = location.longitude

# create map of North York using latitude and longitude values
map_NorthYork = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(ny_data['Latitude'], ny_data['Longitude'], df['Borough'], ny_data['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=True).add_to(map_NorthYork)  
    
map_NorthYork


### Define Foursquare credentials:

In [17]:
CLIENT_ID = 'VZ3YOHRX44O4NYLOUDUUKVHKP1VSZRBJWQNEBWAUXWSLMZNR' # your Foursquare ID
CLIENT_SECRET = '5FEODWZZY32QK41CN0USSL3UBZNVOEBN43ILYUS4Y1Y3I4GX' # your Foursquare Secret
ACCESS_TOKEN = 'ALJCTAFNXUMUZMJMTQ5GHBYLVCUM2RTWLJBQJQADBIXA2WBE'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
print('ACCESS_TOKEN: '+ ACCESS_TOKEN)

Your credentails:
CLIENT_ID: VZ3YOHRX44O4NYLOUDUUKVHKP1VSZRBJWQNEBWAUXWSLMZNR
CLIENT_SECRET:5FEODWZZY32QK41CN0USSL3UBZNVOEBN43ILYUS4Y1Y3I4GX
ACCESS_TOKEN: ALJCTAFNXUMUZMJMTQ5GHBYLVCUM2RTWLJBQJQADBIXA2WBE


### Search for a Specific Venue Type - Pizza place

In [18]:
address = 'North York, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
ny_latitude= location.latitude
ny_longitude = location.longitude

Search for pizza place in North york in 1000metre radius

In [24]:
search_query = 'pizza'
radius = 500
print(search_query + ' .... OK!')

pizza .... OK!


In [25]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, ny_latitude, ny_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VZ3YOHRX44O4NYLOUDUUKVHKP1VSZRBJWQNEBWAUXWSLMZNR&client_secret=5FEODWZZY32QK41CN0USSL3UBZNVOEBN43ILYUS4Y1Y3I4GX&ll=43.7543263,-79.44911696639593&oauth_token=ALJCTAFNXUMUZMJMTQ5GHBYLVCUM2RTWLJBQJQADBIXA2WBE&v=20180605&query=pizza&radius=500&limit=100'

In [26]:
results = requests.get(url).json()



Exctract pizza place information from json file 

In [27]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

<ipython-input-27-67768799b19f>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,50f9bbcc5d24acebc25936af,Domino's Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1624226861,False,820 Sheppard Ave W,43.753127,-79.450926,"[{'label': 'display', 'lat': 43.75312660212406...",197,M3H 2T1,CA,Toronto,ON,Canada,"[820 Sheppard Ave W, Toronto ON M3H 2T1, Canada]"
1,4eea53e2754a186843dd5c77,Double Double Pizza and Chicken,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1624226861,False,871 Sheppard Ave W,43.752048,-79.454026,"[{'label': 'display', 'lat': 43.75204769507766...",469,M3H,CA,North York,ON,Canada,"[871 Sheppard Ave W, North York ON M3H, Canada]"


### Filter the dataframe with information required:

In [28]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Domino's Pizza,Pizza Place,820 Sheppard Ave W,43.753127,-79.450926,"[{'label': 'display', 'lat': 43.75312660212406...",197,M3H 2T1,CA,Toronto,ON,Canada,"[820 Sheppard Ave W, Toronto ON M3H 2T1, Canada]",50f9bbcc5d24acebc25936af
1,Double Double Pizza and Chicken,Pizza Place,871 Sheppard Ave W,43.752048,-79.454026,"[{'label': 'display', 'lat': 43.75204769507766...",469,M3H,CA,North York,ON,Canada,"[871 Sheppard Ave W, North York ON M3H, Canada]",4eea53e2754a186843dd5c77


There's only two pizza places in 500 metre radius.

### Lets Visualise the Pizza places nearby:

In [29]:
venues_map = folium.Map(location=[ny_latitude, ny_longitude], zoom_start=13) # generate map centred around North York
# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [ny_latitude, ny_longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

##  Explore Downtown toronto

In [31]:
dt_data = df[df['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
dt_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
1,M7A,Downtown Toronto,Queen's Park,43.6641,-79.3889
2,M5B,Downtown Toronto,"Ryerson, Garden District",43.6572,-79.3783
3,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756
4,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754


In [32]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
dt_latitude= location.latitude
dt_longitude = location.longitude

In [33]:
  search_query = 'pizza'
radius = 500
print(search_query + ' .... OK!')

pizza .... OK!


In [34]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dt_latitude, dt_longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=VZ3YOHRX44O4NYLOUDUUKVHKP1VSZRBJWQNEBWAUXWSLMZNR&client_secret=5FEODWZZY32QK41CN0USSL3UBZNVOEBN43ILYUS4Y1Y3I4GX&ll=43.6563221,-79.3809161&oauth_token=ALJCTAFNXUMUZMJMTQ5GHBYLVCUM2RTWLJBQJQADBIXA2WBE&v=20180605&query=pizza&radius=500&limit=100'

In [35]:
results = requests.get(url).json()
#results

In [37]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-37-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4b2438f6f964a520126424e3,Pizza Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1624227468,False,260 CHURCH STREET,43.656513,-79.377242,"[{'label': 'display', 'lat': 43.65651263631174...",296,M5B 1Z2,CA,Toronto,ON,Canada,"[260 CHURCH STREET, Toronto ON M5B 1Z2, Canada]",NaN,NaN
1,4af5d885f964a520b2fd21e3,Amato Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1624227468,False,429A Yonge St,43.660215,-79.382571,"[{'label': 'display', 'lat': 43.66021482917061...",453,NaN,CA,Toronto,ON,Canada,"[429A Yonge St (at College St), Toronto ON, Ca...",at College St,NaN
2,5615b6c4498e3c32c67ad78f,Blaze Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1624227468,False,"10 Dundas Street East, #124",43.656518,-79.380015,"[{'label': 'display', 'lat': 43.656518, 'lng':...",75,M5B 2G9,CA,Toronto,ON,Canada,"[10 Dundas Street East, #124, Toronto ON M5B 2...",NaN,NaN
3,4ce805f68ef78cfa1675979b,Pizza Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1624227468,False,346 YONGE STREET,43.657879,-79.381729,"[{'label': 'display', 'lat': 43.65787913074364...",185,M5B 1R8,CA,Toronto,ON,Canada,"[346 YONGE STREET, Toronto ON M5B 1R8, Canada]",NaN,NaN
4,5c53a0aea2a6ce002c9e3806,Pizza Shab,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1624227468,False,NaN,43.656972,-79.378484,"[{'label': 'display', 'lat': 43.656972, 'lng':...",208,M5B 1Y2,CA,Toronto,ON,Canada,"[Toronto ON M5B 1Y2, Canada]",NaN,NaN


In [38]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Pizza Pizza,Pizza Place,260 CHURCH STREET,43.656513,-79.377242,"[{'label': 'display', 'lat': 43.65651263631174...",296,M5B 1Z2,CA,Toronto,ON,Canada,"[260 CHURCH STREET, Toronto ON M5B 1Z2, Canada]",NaN,4b2438f6f964a520126424e3
1,Amato Pizza,Pizza Place,429A Yonge St,43.660215,-79.382571,"[{'label': 'display', 'lat': 43.66021482917061...",453,NaN,CA,Toronto,ON,Canada,"[429A Yonge St (at College St), Toronto ON, Ca...",at College St,4af5d885f964a520b2fd21e3
2,Blaze Pizza,Pizza Place,"10 Dundas Street East, #124",43.656518,-79.380015,"[{'label': 'display', 'lat': 43.656518, 'lng':...",75,M5B 2G9,CA,Toronto,ON,Canada,"[10 Dundas Street East, #124, Toronto ON M5B 2...",NaN,5615b6c4498e3c32c67ad78f
3,Pizza Pizza,Pizza Place,346 YONGE STREET,43.657879,-79.381729,"[{'label': 'display', 'lat': 43.65787913074364...",185,M5B 1R8,CA,Toronto,ON,Canada,"[346 YONGE STREET, Toronto ON M5B 1R8, Canada]",NaN,4ce805f68ef78cfa1675979b
4,Pizza Shab,Pizza Place,NaN,43.656972,-79.378484,"[{'label': 'display', 'lat': 43.656972, 'lng':...",208,M5B 1Y2,CA,Toronto,ON,Canada,"[Toronto ON M5B 1Y2, Canada]",NaN,5c53a0aea2a6ce002c9e3806
5,Boston Pizza,Pizza Place,407 Yonge St,43.659338,-79.382260,"[{'label': 'display', 'lat': 43.65933831769298...",352,M5B 1S9,CA,Toronto,ON,Canada,"[407 Yonge St, Toronto ON M5B 1S9, Canada]",NaN,59ecdaae029a5529f5e8773c
6,Vinnie's Pizza & Pasta,Pizza Place,595 Bay St.,43.656212,-79.383430,"[{'label': 'display', 'lat': 43.65621225108779...",202,M5G 2M6,CA,Toronto,ON,Canada,"[595 Bay St. (in Atrium On Bay), Toronto ON M5...",in Atrium On Bay,4b7ddd2cf964a52001d82fe3
7,Colombo's Pizza,Food Truck,NaN,43.658333,-79.379135,"[{'label': 'display', 'lat': 43.65833282470703...",265,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4ffc93cce4b001c1ca261790
8,Pizza Pizza,Pizza Place,761 Bay St,43.659487,-79.385207,"[{'label': 'display', 'lat': 43.65948714793604...",493,M5G 2R2,CA,Toronto,ON,Canada,"[761 Bay St, Toronto ON M5G 2R2, Canada]",NaN,4bca3d4cb6c49c74e7209091
9,Mamma's Pizza,Pizza Place,Yonge St.,43.659155,-79.383018,"[{'label': 'display', 'lat': 43.659155, 'lng':...",357,NaN,CA,Toronto,ON,Canada,"[Yonge St., Toronto ON, Canada]",NaN,524a5b79498e2a7a5945a8e1


In [39]:
dataframe_filtered.name

0                                Pizza Pizza
1                                Amato Pizza
2                                Blaze Pizza
3                                Pizza Pizza
4                                 Pizza Shab
5                               Boston Pizza
6                     Vinnie's Pizza & Pasta
7                            Colombo's Pizza
8                                Pizza Pizza
9                              Mamma's Pizza
10                          Fantastico Pizza
11    Classic italian style pizza food truck
12                          Bocconcini Pizza
13                            Andrew's Pizza
14                         Slice of New York
15                                 Pizzaiolo
16                                    Panago
17                                 Pizzaiolo
Name: name, dtype: object

There are 17 pizza places in Downtown Toronto

### Lets Visualise the Pizza shops in Downtown Toronto

In [42]:
venues_map = folium.Map(location=[dt_latitude, dt_longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [dt_latitude, dt_longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map